In [6]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
import re


files = ["Episode_I.txt", "Episode_II.txt", "Episode_III.txt", "EIV_Script.txt", "Episode_V.txt", "Episode_VI.txt"]


# Not a super nice way to do this, but you can also try to use this code in conjunction with get_chars to get more characters in a scene
# It just matches for capital words with spaces in scenes, which unfortunately also gets scene descriptions :(
def get_characters(str):
    chars = set()
    # match = re.findall(r'(\s+\/*)([A-Z]{3,}(\s[A-Z]+)*)', str) 
    match = re.findall(r'(\s+\/*)([A-Z]{3,}([^\n\:\.\,]\s{0,1}[A-Z]+)*)', str)
    if(len(match) > 0):
        for m in match:
            for item in m:
                if(len(item.strip().title()) > 1):
                    chars.add(item.strip().title())
    return chars

def get_chars(scene):
    characters = get_characters(scene)
    nltk_results = ne_chunk(pos_tag(word_tokenize(scene)))
    for nltk_result in nltk_results:
        if type(nltk_result) == Tree and nltk_result.label() == "PERSON":
            name = ''
            for nltk_result_leaf in nltk_result.leaves():
                name += nltk_result_leaf[0] + ' '
            characters.add(name.strip())
    return characters

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\timjh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\timjh\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\timjh\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [7]:
import collections
# def merge_dicts(*dicts):
#     res = collections.defaultdict(list)
#     for d in dicts:
#         for k, v in d.iteritems():
#             res[k].append(v)
#     return res

def merge_dicts(*dicts):
    d = {}
    for dict in dicts:
        for key in dict:
            try:
                for item in dict[key]:
                    if item not in d[key]:
                        d[key].append(item)
                #d[key].append(dict[key])
            except KeyError:
                d[key] = dict[key]
    return d

In [18]:
def resolve_characters(set_people):
    # ALL ALIASES FOR ALL 6 MOVIES
    all_people = {
    'Obi-Wan Kenobi': ['Obi-Wan', 'Kenobi', 'Master Jedi', 'Obi-Wan Kenobi', 'Master Obi-Wan Kenobi', 'Master Kenobi', 'Obi Wan', 'Old Ben', 'Ben Kenobi', 'Old Man', 'Ben'], 
    'Anakin Skywalker': ['boy', 'Anakin', 'Anakin Skywalker', 'Skywalker', 'Annie', 'Anakin SKywalker', 'Young Skywalker', 'my young padawan', 'Master Skywalker', 'Master Anakin', 'Helloo Annie'],
    'Jar Jar Binks': ['Jar Jar', 'Binks', 'Jar Jar Binks', 'Jaja Binkss', 'Bombad general', 'Binkss'], 
    'Nute Gunray': ['Viceroy', 'Gunray', 'Nute', 'Nute Gunray', 'Viceroy Gunray'], 
    'Padme Amidala': ['Padme Naberrie', 'Padme Amidala', 'Padme', 'Miss Padme', 'Padme Amidala', 'Senator Amidala', 'My Lady', 'Amidala', 'Miss Padme', 'Queen Amidala', 'Queen', 'Amidala', 'Sabe', 'Sadbe', 'Your Majesty'], 
    'Yoda': ['Master Yoda', 'Yoda', 'creature', 'Jedi Master'], 
    'Mace Windu': ['Master Windu', 'Mace', 'Windu', 'Mace Windu'], 
    'Battle Droid': ['battle droid', 'battle droids', 'OOM-9', 'Droids', 'droid sergeant', 'battle droid captain', 'guard droid', 'Roger', 'command droids'], 
    'Watto': ['Watto'], 
    'Palpatine': ['Senator Palpatine', 'Darth Sidious', 'Palpatione', 'Sidious', 'Palpatine', 'Chancellor Palpatine', 'My Lord', 'Sith Lord', 'Chancellor', 'Sldious', 'Darth Sldious', 'Emperor', 'Lord Sidious'], 
    'Sio Bibble': ['Bibble', 'Sio Bibble'], 
    'R2-D2': ['Artoo Detoo', 'Artoo', 'R2-D2', 'little blue droid', 'Astra droid', 'Astro droid', 'Artoo Deetoo', 'Little Artoo', 'Nearby Artoo'], 
    'C-3PO': ['See Threepio', 'Threepio', 'C-3PO','Seethreepiowha', 'C-3P0'], 
    'Shmi': ['Shmi', 'Shmi Skywalker'], 
    'Bail Organa': ['Bail Organa', 'Prince Bail Organa', 'Bail', 'Bail Antilles', 'Senator Organa'], 
    'Ki-Adi-Mundi': ['Ki-Adi-Mundi', 'Ki-Adi'], 'Mas Amedda': ['Vice Chairman Mas Amedda', 'Mas Amedda'], 
    'Destroyer Droids': ['Droidekas', 'Destroyer Droids', 'Destroyers'], 
    'Cliegg Lars': ['Cliegg Lars', 'Cliegg'], 
    'Owen Lars': ['Owen', 'Owen Lars', 'Master Owen', 'Uncle Owen'], 
    'Beru Lars': ['Beru', 'Beru Whitesun', 'Aunt Beru', 'Beru Lars'], 
    'Sifo-Dyas': ['Master Sifo-Dyas', 'Sifo-Dyas', 'Sido-Dyas'], 
    'Count Dooku': ['Tyranus', 'Count Dooku', 'Dooku', 'Darth Tyranus', 'Lord Tyranus', 'Count'], 
    'Jango Fett': ['Jango Fett', 'Jango', 'Armoured Rocket Man'], 
    'Zam Wessel': ['Zam', 'Zam Wessel', 'Bounty Hunter'], 
    'Boba Fett': ['Boba', 'Boba Fett', 'bounty hunter', 'Fett'], 
    'Tusken Raider': ['Sand People', 'Tuskens', 'Tusken Raider', 'Tusken', 'Raider', 'Tusken Raiders', 'Tusekn Raiders', 'Tusken Child'],
    'Dorme': ['Dorme'], 
    'Captain Typho': ['Captain Typho', 'Typho'], 
    'Corde': ['Corde', 'Cord'], 
    'Ask Aak': ['Senator Ask Aak', 'Ask Aak'], 
    'Darsana': ['Darsana'], 
    'Orn Free Taa': ['Orn Free Taa'],
    'Plo Koon': ['Plo Koon', 'Plot Koon', 'Plo'],
    'Royal Guards': ['Royal Guards', 'Imperial Guards', 'red-cloaked guards'],
    'Dar Wac': ['Dar Wac'],
    'Elan Sleazebaggano': ['Elan Sleazebaggano', 'Elan'],
    'PK-4': ['PK-4'],
    'Dexter Jettster': ['Dex', 'Dexter', 'Dexter Jettster'], 
    'Queen Jamillia': ['Queen Jamillia', 'Your Highness', 'Her Highness'],
    'Jocasta Nu': ['Jocasta Nu'], 
    'Ryoo': ['Ryoo'], 
    'Pooja': ['Pooja'], 
    'Sola': ['Sola'], 
    'Jobal': ['Jobal Naberrie', 'Jobal'], 
    'Ruwee': ['Ruwee Naberrie', 'Ruwee'], 
    'Jedi Child Jack': ['Jedi Child Jack'], 
    'Jedi Child May': ['Jedi Child May'], 
    'Taun We': ['Taun We'], 
    'Lama Su': ['Lama Su'], 'R4-D17': ['Arfour', 'R4 D17', 'R4-D17'], 'Pit Droid': ['Pit Droid'],
    'Clone Troopers': ['Clones', 'Clone Trooper', 'Clone Captain', 'Clone', 'Clone Troopers', 'Clone Commander'], 
    'Geonosian': ['Geonosian', 'Geonosians'], 'Poggle the Lesser': ['Poggle', 'Poggle the Lesser'], 
    'Sun Rit': ['Sun Rit'], 'Po Nudu': ['Po Nudu'], 'Tessek': ['Tessek'], 
    'Toonbuck Toora': ['Toonbuck Toora'], 
    'Shu Mai': ['Shu Mai'], 'Passel Argente': ['Passel Argente'], 
    'Wat Tambor': ['Wat Tambor'], 'San Hill': ['San Hill'], 
    'Super Battle Droids': ['superdroids', 'super battle droid'], 
    'Youngling': ['Youngling', 'young one'], 'Qui-Gon Jinn': ['Qui-Gon', 'Master Qui-Gon', 'Qui-Gon Jinn', 'Qui'], 
    'TC-14': ['TC-14'], 'Jabba the Hutt': ['Jabba', 'Jabba the Hutt', 'Jabba Du Hutt', 'Hutt'], 
    'Captain Tarpals': ['Captain Tarpals', 'Tarpals'], 'Boss Nass': ['Boss', 'Boss Nass'], 
    'Republic Captain': ['Captain'], 'Captain Panaka': ['Captain Panaka', 'Panaka', 'Capt. Panaka', 'Captain Pnaka'], 
    'Darth Maul': ['Darth Maul', 'Maul', 'Sith apprentice', 'Sith assassin', 'Lord Maul'], 
    'Dofine': ['Dofine', 'Captain Dofine'], 'Rune': ['Rune'], 'Tey How': ['Tey How'], 
    'Sebulba': ['Sebulba'], 'Eirtae': ['Eirtae'], 'Yane': ['Yane'], 'Rabe': ['Rabe'], 
    'Sache': ['Sache'], 'Chancellor Valorum': ['Supreme Chancellor', 'Chancellor Valorum', 'Valorum', 'Spreme Chancellor'], 
    'Ric Olie': ['Ric Olie'], 
    'Jira': ['Jira'], 
    'Kitster': ['Kitster'], 'Seek': ['Seek'], 'Amee': ['Amee'], 'Wald': ['Wald'], 
    'Fode/Beed': ['Fode/Beed', 'Fode Beed'], 'Ben Quadrinaros': ['Ben Quadrinaros', 'Quadrinaros', 'Quadinaros'], 
    'Boles Roor': ['Boles Roor'], 'Gasgano': ['Gasgano'], 'Mawhonic': ['Mawhonic'], 'Gardulla': ['Gardulla', 'Gardulla the Hutt'], 
    'Xelbree': ['Xelbree'], 'Fanta': ['Fanta'], 'Ody Mandrell': ['Ody', 'Ody Mandrell'], 
    'Terter': ['Terter'], 'Sith Probe Droid': ['Sith Probe Droid', 'Probe Droid'], 
    'Greedo': ['Greedo'], 'Lott Dod': ['Lott Dod'], 'Aks Moe': ['Aks Moe'], 
    'Gungan Warrior': ['Gungan lookout', 'Gungan sentry', 'Gungans'], 'Bravo Two': ['Bravo Two'], 
    'Bravo Three': ['Bravo Three'], 
    'General Grievous': ['General Grievous', 'Grievous'], 'Buzz Droid': ['Buzz Droid'], 'Odd Ball': ['Odd Ball'], 
    'Clone Pilot': ['Clone Pilot'], 'Nemoidian Captain': ['Nemoidian Captain'], 'Bodyguard': ['Bodyguard', 'Magnaguard'], 
    'Fireship Pilot': ['Fireship Pilot'], 
    'Luke Skywalker': ['Luke', 'baby', 'Luke Skywalker', 'twins', 'belly', 'kid', 'Master Luke', 'Red Five', 'Skywalker', 'boy', 'Rogue leader Luke', 'Rogue leader', 'Opposite Luke', 'Again Luke', 'Jedi Knight'], 
    'Princess Leia': ['baby', 'Leia', 'Leia Organa', 'twins', 'belly', 'Princess Leia', 'Your Worshipfulness', 'Princess', 'your Worship', 'Boushh'], 
    'Master Vos': ['Master Vos', 'Quinlan Vos'], 'Mon Mothma': ['Mon Mothma', 'Mon', 'Mothma'], 'Fang Zar': ['Fang Zar'], 
    'Terr Taneel': ['Terr Taneel'], 'Giddean Danu': ['Giddean Danu'], 'Eeth Koth': ['Eeth Koth'], 'Elle': ['Elle'], 'Motee': ['Motee'], 'Sly-Moore': ['Sly-Moore'], 'Darth Plagueis': ['Darth Plagueis'], 'Agen Kolar': ['Agen Kolar'], 
    'Chewbacca': ['Chewbacca', 'Chewie', 'Wookie'], 'Wookie Warrior': ['Wookie chieftan', 'Wookie army', 'Wookie', 'Wookie warrior'], 'Bana Breemu': ['Bana Breemu'], 'Chi Eekway': ['Chi Eekway'], 'Clone Commander Cody': ['Clone Commander Cody', 'Cody'], 'Tion Medon': ['Tion Medon', 'Tlon Medon'], 'R4-G9': ['R4-G9', 'Geenine'], 'Wrangler': ['Wrangler'], 'Rune Haako': ['Rune Haako'], 'Nee Avalar': ['Nee Avalar'], 'Maledee': ['Maledee'], 'Sweitt Concorkill': ['Sweitt Concorkill'], 'Aayla Secura': ['Aayla Secura', 'Aayla'], 'Saesee Tinn': ['Saesee Tinn'], 'Kit Fisto': ['Kit Fisto'], 
    'Darth Vader': ['Lord Vader', 'Vader', 'Darth Vader', 'Darth', 'Father', 'Piett Lord Vader'], 
    'Clone Commander Bacara': ['Clone Commander Bacara'], 'Barriss Offee': ['Barriss Offee'], 
    'Stass Allie': ['Stass Allie'], 'Clone Commander Gree': ['Clone Commander Gree'], 
    'Tarfful': ['Tarfful'], 'Zett Jukassa': ['Zett Jukassa'], 'Clone Commander Appo': ['Clone Commander Appo'], 
    'Seeker Drone': ['Seeker Drone'], 'AT-ST Clone Sergeant': ['AT-ST Clone Sergeant'], 
    'Captain Antilles': ['Captain Antilles', 'Antilles'], 'Alderaan Trooper': ['Alderaan Trooper'], 
    'Clone Commander Thire': ['Clone Commander Thire', 'Commander Thire'], 'Worker Droid': ['Worker Droid'], 
    'Medical Droid': ['Medical Droid', 'medical droid'], 'Polis Medic': ['Polis Medic'], 
    'Tarkin': ['Governor Tarkin', 'Tarkin', 'Grand Moff Tarkin'], 'Queen of Alderaan': ['Queen of Alderaan'], 
    'Shaak Ti': ['Shaak Ti'], 'Baron Papanoida': ['Baron Papanoida', 'Papanoida'], 
    'Biggs Darklighter': ['Biggs', 'Biggs Darklighter', 'Red Three'], 
    'Han Solo': ['Han', 'Solo', 'Han Solo', 'Captain Solo', 'Scoundrel', 'Again Han'], 
    'Jawa': ['Jawa', 'Jawas'], 'Red Seven': ['Red Seven'], 'Wedge Antilles': ['Wedge', 'Wedge Antilles', 'Red Two', 'Rogue Three', 'Red Leader'], 'Red Ten': ['Red Ten'], 'Deak': ['Deak'], 'General Dodonna': ['General Dodonna', 'Dodonna'], 'Wormie': ['Wormie'], 'Gold Five': ['Gold Five'], 'Red Leader': ['Red Leader'], 'Rebel Officer': ['Rebel Officer', 'Rebel lieutenant', 'deck officer', 'senior controller', 'controller', 'Rebel Captain', 'trench officer'], 'Camie': ['Camie'], 
    'Stormtrooper': ['stormtrooper', 'stormtroopers', "man's voice", "Stormtrooper", "Stormtroopers"], 'Fixer': ['Fixer'], 'Windy': ['Windy'], 'Rebel Trooper': ['Rebel', 'Rebels', 'Rebel Trooper', 'Rebel Troops', 'Rebel Fighter', 'announcer'], 'Porkins': ['Porkins', 'Red Six'], 'Red Nine': ['Red Nine'], 'Red Eleven': ['Red Eleven'], 'Admiral Motti': ['Admiral Motti', 'Motti'], 'Commander Tagge': ['Commander Tagge', 'Tagge'], 'Bartender': ['Bartender'], 'Ponda Babba': ['Creature'], 'Doctor Evezan': ['Human'], 'Imperial Officer': ['Imperial Officer', 'commander', 'bunker commander', 'Imperial soldier', 'Imperial controller'], 'Gold Two': ['Gold Two'], 'Lando Calrissian': ['Lando', 'Lando Calrissian', 'Calrissian', 'Gold Leader', 'administrator', 'Lando Lando'], 
    'Admiral Piett': ['Admiral Piett', 'Piett', 'Captain Piett', 'Piett Yes', 'Piett Admiral', 'Piett bounty', 'Piett Thank', 'Piett Right', 'Piett Alert', 'PIETT Hold', 'PIETT Shall', 'PIETT Intensify'], 'Rebel Pilot': ['Rebel Pilot', 'Y-Wing Pilot', 'Pilot', 'X-wings pilots'], 'Captain Needa': ['Captain Needa', 'Needa'], 'Major Derlin': ['Derlin', 'Major Derlin', 'Major'], 'Janson': ['Janson', 'Janson Cable'], 'Zev': ['Rogue Two', 'Zev'], 'Dack Ralter': ['Dack Ralter', 'Dack'], 'General Rieekan': ['Rieekan', 'General Rieekan'], 'Imperial Probe Droid': ['probe robot', 'probe droid'], 'Admiral Ozzel': ['Ozzel', 'Admiral Ozzel'], 'General Veers': ['Veers', 'General Veers'], 'Hobbie': ['Hobbie'], 'Walker Pilot': ['walker pilot', 'Walker Pilot'], 'IG-88': ['IG-88'], 'Dengar': ['Dengar'], 'Zuckuss': ['Zuckuss'], 'Bossk': ['Bossk'], '4-LOM': ['bounty hunters'], 'Lobot': ['Lobot'], 'Ugnaught': ['Ugnaught'], 
    'Bespin Guards': ['Bespin Guards'], 'Logray': ['Logray'], 'Chief Chirpa': ['Chief Chirpa'], 'Ewoks': ['Ewoks'], 'Wicket': ['Wicket'], 'Nien Nunb': ['Nien Nunb'], 'General Madine': ['Madine', 'General Madine'], 'Paploo': ['Paploo'], 'Admiral Ackbar': ['Ackbar', 'Admiral Ackbar'], 'Bib Fortuna': ['Bib', 'Bib Fortuna', 'Fortuna'], 'Shuttle Captain': ['Shuttle Captain'], 'Moff Jerjerrod': ['Jerjerrod', 'Moff Jerjerrod'], 'Gamorrean Guard': ['Gamorrean Guard', 'Gamorrean'], 'Salacious Crumb': ['Salacious Crumb', 'Crumb', 'Salacious'], 'Oola': ['Oola'], 'EV-9D9': ['EV-9D9', 'Ninedenine'], 'Rancor Keeper': ['Keeper'], 'Flight Controller': ['Controller'], 'Imperial Scout Trooper': ['Imperial scouts', 'scout', 'biker scouts'], 'Teebo': ['Teebo'], 'Gray Leader': ['Grey Leader'], 'Green Leader': ['Green Leader'], 'Red Two': ['Red Two'], 'Red Three': ['Red Three']}


    return_set = set()
    for item in all_people:
        for thing_string in set_people:
            arr = all_people[item]
            if thing_string in arr:
                return_set.add(item)
    return return_set
    

# ALL CHARACTERS FROM ALL 6 MOVIES
set_people = {'Green Leader', 'Qui-Gon Jinn', 'Alderaan Trooper', 'Chi Eekway', 'Oola', 'Sifo-Dyas', 'Yoda', 'Royal Guards', 'Fang Zar', 'Owen Lars', 'Red Two', 'Dexter Jettster', 'Tion Medon', 'Captain Antilles', 'Rebel Pilot', 'Admiral Piett', 'Nee Avalar', 'Wrangler', 'Imperial Officer', 'Cliegg Lars', 'R2-D2', 'TC-14', 'AT-ST Clone Sergeant', 'Ryoo', 'Flight Controller', 'Boss Nass', 'Ruwee', 'Eirtae', 'Teebo', 'Commander Tagge', 'Red Leader', 'General Dodonna', 'Tey How', 'EV-9D9', 'Biggs Darklighter', 'Fode/Beed', 'Wookie Warrior', 'Major Derlin', 'Agen Kolar', 'Ewoks', 'Windy', 
'Logray', 'Dorme', 'Red Three', 'Bespin Guards', 'Wicket', 'Taun We', 'Wedge Antilles', 'Jango Fett', 'Sith Probe Droid', 'Shu Mai', 'Jawa', 'IG-88', 'Boles Roor', 'Nute Gunray',
'Amee', 'Sola', 'Paploo', 'Princess Leia', 'Bossk', 'Boba Fett', 'Sun Rit', 'Bib Fortuna', 'Saesee Tinn', 'Gray Leader', 'Chief Chirpa', 'Bartender', 'Chancellor Valorum', 'Wald', 'Bana Breemu', 'Rancor Keeper', 'Pit Droid', 'Lott Dod', 'General Rieekan', 'Passel Argente', 'Ugnaught', 'Lando Calrissian', 'Doctor Evezan', 'Admiral Ackbar', 'Obi-Wan Kenobi', 'R4-G9', 'Mon Mothma', 'Eeth Koth', 'Dengar', 'Jar Jar Binks', 'Ben Quadrinaros', 'Toonbuck Toora', 'Palpatine', 'Queen of Alderaan', 'Rabe', 'Pooja', 'Polis Medic', 'Clone Commander Gree', 'Jira', 'Sly-Moore', 'Gasgano', 'Odd Ball', 'Nien Nunb', '4-LOM', 'Sio Bibble', 'Jabba the Hutt', 'Captain Panaka', 'Dar Wac', 'Stass Allie', 'Poggle the Lesser', 'Red Ten', 'Fanta', 'Jabba the HuttCamie', 'Shaak Ti', 'Admiral Ozzel', 'Captain Tarpals', 'Rune', 'Zam Wessel', 'Aayla Secura', 'Sweitt Concorkill', 'Gold Five', 'Medical Droid', 'Super Battle Droids', 'General Grievous', 'Fireship Pilot', 'Zev', 
'Sebulba', 'Aks Moe', 'Darsana', 'Orn Free Taa', 'Clone Pilot', 'Seeker Drone', 'Motee', 'Gungan Warrior', 'Chewbacca', 'Imperial Probe Droid', 'Luke Skywalker', 'Gold Two', 'Nemoidian Captain', 'Kit Fisto', 'Rebel Officer', 'Red Eleven', 'Greedo', 'General Madine', 'General Veers', 'Darth Vader', 'Queen',
'Gardulla', 'Shmi', 'Clone Commander Thire', 'C-3PO', 'Jedi Child Jack', 'Padme Amidala', 'Admiral Motti', 'Mawhonic', 'Tusken Raider', 'Baron Papanoida', 'Walker Pilot', 'Stormtrooper', 'Mace Windu', 'Plo Koon', 'Tessek', 'Bodyguard', 'Clone Troopers', 'Darth Maul', 'Watto', 'Bravo Two', 'Ody Mandrell', 'Dofine', 'Jobal', 'Darth Plagueis', 'San Hill', 'Captain Needa', 'Rebel Trooper', 'Mas Amedda', 'Beru Lars', 'Queen Jamillia', 'Jocasta Nu', 'Barriss Offee', 'Zett Jukassa', 'Xelbree', 'Deak', 'Clone Commander Appo', 'Kitster', 'Red Seven', 'Sache', 'Dack Ralter', 'Rune Haako', 'Hobbie', 'Worker Droid', 'Elle', 'Ponda Babba', 'Clone Commander Cody',
'Clone Commander Bacara', 'Zuckuss', 'Gamorrean Guard', 'Anakin Skywalker', 'Ki-Adi-Mundi', 'Wat Tambor', 'Geonosian', 'Battle Droid', 'Ask Aak', 'Ric Olie', 'Seek', 'Tarkin', 'Fixer', 'Lobot', 'Giddean Danu', 'PK-4', 'Janson', 'Bravo Three', 'Salacious Crumb', 'Republic Captain', 'Lama Su', 'Count Dooku', 'Wormie', 'Bail Organa', 'Buzz Droid', 'Shuttle Captain', 'Youngling', 'Corde', 'Elan Sleazebaggano', 'R4-D17', 'Maledee', 'Tarfful', 'Imperial Scout Trooper', 'Han Solo', 'Moff Jerjerrod', 'Terter', 'Destroyer Droids', 'Captain Typho', 'Jedi Child May', 'Master Vos', 'Terr Taneel', 'Red Nine', 'Po Nudu', 'Yane'}


In [8]:
from collections import defaultdict

def def_value():
    return 0

all_connections = defaultdict(def_value)
all_people = defaultdict(def_value)

# Split a set into all possible subsets of length 2
# Because we will later use these sets as keys, they must be returned as frozensets which are immutable
def split_set(data):
    d_list = list(data)
    return list([frozenset((a, b)) for idx, a in enumerate(d_list) for b in d_list[idx + 1:]])

# Returns a defaultdict of type set:int representing two characters in a set with the number of their occurrances together,
# given a list of sets
def group_chars(data):
    for item in data:
        item = frozenset(item)
        if len(item) > 2:
            subsets = split_set(item)
            for subset in subsets:
                all_connections[subset] += 1
        if len(item) == 2:
            all_connections[item] += 1

# Same method as above, but works on only one set of characters
def group_chars_single(dict, item):
    item = frozenset(item)
    if len(item) > 2:
        subsets = split_set(item)
        for subset in subsets:
            dict[subset] += 1
    if len(item) == 2:
        dict[item] += 1

# Returns a dictionary of string:int representing how many scenes a character is in
def count_chars(data):
    for item in data:
        for char in item:
            all_people[char] += 1


#print(all_connections) # all_connections holds a key of two characters as a frozenset, and a value of their occurrances together in scenes

In [16]:
all_connections = defaultdict(def_value)
all_people = defaultdict(def_value)


for filename in files:
    file = open(filename, encoding="utf-8")
    script = file.read()

    script = script.replace("EXT.", "INT.")
    script = script.replace(r'(INT.).*', "INT.")
    scenes = script.split("INT.")

    for scene in scenes:
        chars = get_chars(scene)
        if len(chars) != 0:
            cc = resolve_characters(chars)
            group_chars_single(all_connections, cc)
            for char in cc:
                all_people[char] += 1 # Append to running total of all characters in all scenes
            

In [17]:
# Write characters to file - be careful about re-running to avoid duplicate data
with open('characters.csv', 'w') as f:
    f.write("id,count\n")
    for key in all_people.keys():
        f.write("%s,%s\n"%(key,all_people[key]))


# for key in all_connections.keys():
#     print("%s,%s\n"%("?".join(key),all_connections[key]))

with open('occs.csv', 'w') as f:
    f.write("source,target,total\n")
    for key in all_connections.keys():
        f.write("%s,%s\n"%(",".join(key),all_connections[key]))


In [61]:
# Print sorted occurrence counts for each character
import operator
sorted(all_people.items(), key=operator.itemgetter(1))


[('Gardulla', 1),
 ('Seek', 1),
 ('Amee', 1),
 ('Boles Roor', 1),
 ('Mawhonic', 1),
 ('Xelbree', 1),
 ('Terter', 1),
 ('Aks Moe', 1),
 ('Lott Dod', 1),
 ('Corde', 1),
 ('Darsana', 1),
 ('Elan Sleazebaggano', 1),
 ('Dexter Jettster', 1),
 ('Jocasta Nu', 1),
 ('Ryoo', 1),
 ('Pooja', 1),
 ('Jedi Child May', 1),
 ('Jedi Child Jack', 1),
 ('Sifo-Dyas', 1),
 ('Tessek', 1),
 ('San Hill', 1),
 ('Toonbuck Toora', 1),
 ('Buzz Droid', 1),
 ('Terr Taneel', 1),
 ('Motee', 1),
 ('Darth Plagueis', 1),
 ('Baron Papanoida', 1),
 ('Chi Eekway', 1),
 ('Bana Breemu', 1),
 ('R4-G9', 1),
 ('Wrangler', 1),
 ('Sweitt Concorkill', 1),
 ('Maledee', 1),
 ('Clone Commander Bacara', 1),
 ('Barriss Offee', 1),
 ('Stass Allie', 1),
 ('Shaak Ti', 1),
 ('Clone Commander Appo', 1),
 ('Zett Jukassa', 1),
 ('Royal Guards', 1),
 ('Clone Commander Thire', 1),
 ('Worker Droid', 1),
 ('Windy', 1),
 ('Wormie', 1),
 ('Commander Tagge', 1),
 ('Doctor Evezan', 1),
 ('Bartender', 1),
 ('Red Seven', 1),
 ('Red Nine', 1),
 ('Porkin

# Data for aggregation from all 6 movies

In [13]:
e12_people = {
    "Obi-Wan Kenobi": ["Obi-Wan", "Kenobi", "Master Jedi", "Obi-Wan Kenobi", "Master Obi-Wan Kenobi", "Master Kenobi"],
    "Anakin Skywalker": ["boy", "Anakin", "Anakin Skywalker", "Skywalker", "Annie", "my young padawan", "Master Annie", "Master Anakin", "Anne"],
    "Jar Jar Binks": ["Jar Jar", "Binks", "Jar Jar Binks", "Jaja", "Jar"],
    "Nute Gunray": ["Viceroy", "Gunray", "Nute", "Nute Gunray", "Viceroy Gunray"],
    "Padme Amidala": ["Padme Naberrie", "Padme Amidala", "Senator Amidala", "Padm", "Padm Amidala", "Miss Padme", "Amidala"],
    "Yoda": ["Master Yoda", "Yoda"],
    "Mace Windu": ["Master Windu", "Mace", "Windu", "Mace Windu"],
    "Battle Droid": ["battle droid", "battle droids", "OOM-9", "droid sergeant", "battle droid captain", "guard droid", "command droids"],
    "Watto": ["Watto", "Wattos"],
    "Palpatine": ["Chancellor Palpatine", "Darth Sidious", "Sidious", "Sith Lord", "Palpatine", "Chancellor"],
    "Sio Bibble": ["Bibble", "Sio Bibble"],
    "R2-D2": ["Artoo Detoo", "Artoo", "R2-D2", "little blue droid"],
    "C-3PO": ["See Threepio", "Threepio", "C-3PO"],
    "Shmi": ["Shmi", "Mom", "mother", "Shmi Skywalker"],
    "Bail Organa": ["Bail Organa", "Prince Bail Organa"],
    "Ki-Adi-Mundi": ["Ki-Adi-Mundi", "Ki-Adi"],
    "Mas Amedda": ["Vice Chairman Mas Amedda", "Mas Amedda"],
    "Destroyer Droids": ["Droidekas", "Destroyer Droids", "Destroyers"],
    "Cliegg Lars": ["Cliegg Lars", "Cliegg"],
    "Owen Lars": ["Owen", "Owen Lars", "Master Owen"],
    "Beru Lars": ["Beru", "Beru Whitesun"],
    "Sifo-Dyas": ["Master Sifo-Dyas", "Sifo-Dyas", "Sido-Dyas"],
    "Count Dooku": ["Tyranus", "Count Dooku", "Dooku", "Darth Tyranus", "Lord Tyranus"],
    "Jango Fett": ["Jango Fett", "Jango", "Armoured Rocket Man"],
    "Zam Wessel": ["Zam", "Zam Wessel", "Bounty Hunter"],
    "Boba Fett": ["Boba", "Boba Fett"],
    "Tusken Raider": ["Sand People", "Tuskens", "Tusken Raider", "Tusken", "Raider", "Tusken Raiders", "Tusekn Raiders", "Tusken Child"],
    "Dorme": ["Dorme"],
    "Captain Typho": ["Captain Typho", "Typho"],
    "Corde": ["Corde", "Cord"],
    "Ask Aak": ["Senator Ask Aak", "Ask Aak"],
    "Darsana": ["Darsana"],
    "Orn Free Taa": ["Orn Free Taa"],
    "Plo Koon": ["Plo Koon", "Plot Koon"],
    "Royal Guards": ["Royal Guards"],
    "Dar Wac": ["Dar Wac"],
    "Elan Sleazebaggano": ["Elan Sleazebaggano", "Elan"],
    "PK-4": ["PK-4"],
    "Dexter Jettster": ["Dex", "Dexter", "Dexter Jettster"],
    "Queen Jamillia": ["Queen Jamillia", "Your Highness", "Her Highness"],
    "Jocasta Nu": ["Jocasta Nu"],
    "Ryoo": ["Ryoo"],
    "Pooja": ["Pooja"],
    "Sola": ["Sola"],
    "Jobal": ["Jobal Naberrie", "Jobal"],
    "Ruwee": ["Ruwee Naberrie", "Ruwee"],
    "Jedi Child Jack": ["Jedi Child Jack"],
    "Jedi Child May": ["Jedi Child May"],
    "Taun We": ["Taun We"],
    "Lama Su": ["Lama Su"],
    "R4-D17": ["Arfour", "R4 D17", "R4-D17"],
    "Pit Droid": ["Pit Droid"],
    "Clone Troopers": ["Clones", "Clone Trooper", "Clone Captain"],
    "Geonosian": ["Geonosian", "Geonosians"],
    "Poggle the Lesser": ["Poggle", "Poggle the Lesser"],
    "Sun Rit": ["Sun Rit"],
    "Po Nudu": ["Po Nudu"],
    "Tessek": ["Tessek"],
    "Toonbuck Toora": ["Toonbuck Toora"],
    "Shu Mai": ["Shu Mai"],
    "Passel Argente": ["Passel Argente"],
    "Wat Tambor": ["Wat Tambor"],
    "San Hill": ["San Hill"],
    "Super Battle Droids": ["superdroids"],
    "Youngling": ["Youngling"],
    "Qui-Gon Jinn": ["Qui-Gon", "Master Qui-Gon", "Qui-Gon Jinn", "Qui"],
    "TC-14": ["TC-14"],
    "Jabba the Hutt": ["Jabba", "Jabba the Hutt", "Jabba Du Hutt"],
    "Anakin Skywalker": ["boy", "Anakin", "Anakin Skywalker", "Skywalker", "Annie", "Anakin SKywalker", "Young Skywalker"],
    "Jar Jar Binks": ["Jar Jar", "Binks", "Jar Jar Binks", "Jaja Binkss", "Bombad general", "Binkss"],
    "Captain Tarpals": ["Captain Tarpals", "Tarpals"],
    "Boss Nass": ["Boss", "Boss Nass"],
    "Nute Gunray": ["Viceroy", "Gunray", "Nute", "Nute Gunray"],
    "Republic Captain": ["Captain"],
    "Captain Panaka": ["Captain Panaka", "Panaka", "Capt. Panaka", "Captain Pnaka"],
    "Darth Maul": ["Darth Maul", "Maul", "Sith apprentice", "Sith assassin", "Lord Maul"],
    "Padme Amidala": ["Padme Naberrie", "Padme Amidala", "Padme"],
    "Yoda": ["Master Yoda", "Yoda"],
    "Mace Windu": ["Master Windu", "Mace", "Windu", "Mace Windu"],
    "Dofine": ["Dofine", "Captain Dofine"],
    "Battle Droid": ["battle droid", "battle droids", "OOM-9", "droids", "droid sergeant", "battle droid captain", "guard droid", "Roger"],
    "Rune": ["Rune"],
    "Tey How": ["Tey How"],
    "Sebulba": ["Sebulba"],
    "Watto": ["Watto"],
    "Eirtae": ["Eirtae"],
    "Yane": ["Yane"],
    "Rabe": ["Rabe"],
    "Sache": ["Sache"],
    "Palpatine": ["Senator Palpatine", "Darth Sidious", "Palpatione", "Sidious", "Palpatine", "Chancellor Palpatine", "My Lord"],
    "Sio Bibble": ["Bibble", "Sio Bibble"],
    "Chancellor Valorum": ["Supreme Chancellor", "Chancellor Valorum", "Valorum", "Spreme Chancellor"],
    "R2-D2": ["Artoo Detoo", "Artoo", "R2-D2", "little blue droid", "Astra droid", "Astro droid"],
    "C-3PO": ["See Threepio", "Threepio", "C-3PO"],
    "Ric Olie": ["Ric Olie"],
    "Queen": ["Queen Amidala", "Queen", "Amidala", "Sabe", "Sadbe", "Your Majesty"],
    "Jira": ["Jira"],
    "Shmi": ["Shmi", "Mom", "mother", "Shmi Skywalker"],
    "Kitster": ["Kitster"],
    "Seek": ["Seek"],
    "Amee": ["Amee"],
    "Wald": ["Wald"],
    "Fode/Beed": ["Fode/Beed", "Fode Beed"],
    "Ben Quadrinaros": ["Ben Quadrinaros", "Quadrinaros", "Quadinaros", "Ben"],
    "Boles Roor": ["Boles Roor"],
    "Gasgano": ["Gasgano"],
    "Mawhonic": ["Mawhonic"],
    "Bail Organa": ["Bail Organa", "Prince Bail Organa", "Bail", "Organa", "Bail Antilles"],
    "Gardulla": ["Gardulla", "Gardulla the Hutt"],
    "Xelbree": ["Xelbree"],
    "Fanta": ["Fanta"],
    "Ody Mandrell": ["Ody", "Ody Mandrell"],
    "Terter": ["Terter"],
    "Sith Probe Droid": ["Sith Probe Droid", "Probe Droid"],
    "Greedo": ["Greedo"],
    "Ki-Adi-Mundi": ["Ki-Adi-Mundi", "Ki-Adi"],
    "Lott Dod": ["Lott Dod"],
    "Aks Moe": ["Aks Moe"],
    "Mas Amedda": ["Vice Chairman Mas Amedda", "Mas Amedda"],
    "Gungan Warrior": ["Gungan lookout", "Gungan sentry", "Gungans"],
    "Bravo Two": ["Bravo Two"],
    "Bravo Three": ["Bravo Three"],
    "Destroyer Droids": ["Droidekas", "Destroyer Droids", "Destroyers"],
    "Pit Droid": ["Pit Droid"],
    
}

e3_people = {
    "Obi-Wan Kenobi": ["Obi-Wan", "Kenobi", "Obi-Wan Kenobi", "Master Kenobi", "Obi Wan"],
    "Anakin Skywalker": ["boy", "Anakin", "Anakin Skywalker", "Skywalker", "Annie", "my young padawan", "Master Skywalker", "Master Anakin", "Helloo Annie"],
    "Jar Jar Binks": ["Jar Jar", "Binks", "Jar Jar Binks"],
    "Nute Gunray": ["Viceroy", "Gunray", "Nute", "Nute Gunray", "Viceroy Gunray"],
    "Padme": ["Miss Padme", "Padme Amidala", "Senator Amidala", "My Lady", "Amidala", "Miss Padme"],
    "Yoda": ["Master Yoda", "Yoda"],
    "Mace Windu": ["Master Windu", "Mace", "Windu", "Mace Windu"],
    "Battle Droid": ["battle droid", "droid sergeant", "battle droid captain", "guard droid", "command droids"],
    "Palpatine": ["Chancellor Palpatine", "Darth Sidious", "Sidious", "Sith Lord", "Palpatine", "Chancellor", "Sldious", "Darth Sldious", "Emperor", "Lord Sidious"],
    "Sio Bibble": ["Bibble", "Sio Bibble"],
    "R2-D2": ["Artoo Detoo", "Artoo", "R2-D2", "little blue droid", "Astro droid"],
    "C-3PO": ["See Threepio", "Threepio", "C-3PO"],
    "Bail Organa": ["Bail Organa", "Bail", "Organa", "Senator Organa"],
    "Ki-Adi-Mundi": ["Ki-Adi-Mundi", "Ki-Adi"],
    "Mas Amedda": ["Vice Chairman Mas Amedda", "Mas Amedda"],
    "Destroyer Droids": ["Droidekas", "Destroyer Droids", "Destroyers"],
    "Owen Lars": ["Owen", "Owen Lars", "Uncle Owen"],
    "Beru Lars": ["Beru", "Beru Whitesun", "Aunt Beru"],
    "Count Dooku": ["Tyranus", "Count Dooku", "Dooku", "Darth Tyranus"],
    "Captain Typho": ["Captain Typho", "Typho"],
    "Plo Koon": ["Plo Koon", "Plo"],
    "Royal Guards": ["Royal Guards"],
    "Dar Wac": ["Dar Wac"],
    "Queen Jamillia": ["Queen Jamillia", "Your Highness"],
    "R4-D17": ["Arfour", "R4 D17", "R4-D17"],
    "Clone Troopers": ["Clone", "Clone Troopers", "Clone Captain", "Clone Commander"],
    "Poggle the Lesser": ["Poggle", "Poggle the Lesser"],
    "Sun Rit": ["Sun Rit"],
    "Po Nudu": ["Po Nudu"],
    "Tessek": ["Tessek"],
    "Toonbuck Toora": ["Toonbuck Toora"],
    "Shu Mai": ["Shu Mai"],
    "Passel Argente": ["Passel Argente"],
    "Wat Tambor": ["Wat Tambor"],
    "San Hill": ["San Hill"],
    "Super Battle Droids": ["superdroids", "super battle droid"],
    "General Grievous": ["General Grievous", "Grievous"],
    "Buzz Droid": ["Buzz Droid"],
    "Odd Ball": ["Odd Ball"],
    "Clone Pilot": ["Clone Pilot"],
    "Nemoidian Captain": ["Nemoidian Captain"],
    "Bodyguard": ["Bodyguard", "Magnaguard"],
    "Fireship Pilot": ["Fireship Pilot"],
    "Luke Skywalker": ["Luke", "baby", "Luke Skywalker", "twins", "belly"],
    "Princess Leia": ["baby", "Leia", "Leia Organa", "twins", "belly"],
    "Master Vos": ["Master Vos", "Quinlan Vos"],
    "Mon Mothma": ["Mon Mothma", "Mon"],
    "Fang Zar": ["Fang Zar"],
    "Terr Taneel": ["Terr Taneel"],
    "Giddean Danu": ["Giddean Danu"],
    "Eeth Koth": ["Eeth Koth"],
    "Elle": ["Elle"],
    "Motee": ["Motee"],
    "Sly-Moore": ["Sly-Moore"],
    "Darth Plagueis": ["Darth Plagueis"],
    "Agen Kolar": ["Agen Kolar"],
    "Chewbacca": ["Chewbacca", "Chewie"],
    "Wookie Warrior": ["Wookie chieftan", "Wookie army", "Wookie", "Wookie warrior"],
    "Bana Breemu": ["Bana Breemu"],
    "Chi Eekway": ["Chi Eekway"],
    "Clone Commander Cody": ["Clone Commander Cody", "Cody"],
    "Tion Medon": ["Tion Medon", "Tlon Medon"],
    "R4-G9": ["R4-G9", "Geenine"],
    "Wrangler": ["Wrangler"],
    "Rune Haako": ["Rune Haako"],
    "Nee Avalar": ["Nee Avalar"],
    "Maledee": ["Maledee"],
    "Sweitt Concorkill": ["Sweitt Concorkill"],
    "Aayla Secura": ["Aayla Secura", "Aayla"],
    "Saesee Tinn": ["Saesee Tinn"],
    "Kit Fisto": ["Kit Fisto"],
    "Darth Vader": ["Lord Vader", "Vader", "Darth Vader"],
    "Clone Commander Bacara": ["Clone Commander Bacara"],
    "Barriss Offee": ["Barriss Offee"],
    "Stass Allie": ["Stass Allie"],
    "Clone Commander Gree": ["Clone Commander Gree"],
    "Tarfful": ["Tarfful"],
    "Youngling": ["Youngling", "young one"],
    "Zett Jukassa": ["Zett Jukassa"],
    "Clone Commander Appo": ["Clone Commander Appo"],
    "Seeker Drone": ["Seeker Drone"],
    "AT-ST Clone Sergeant": ["AT-ST Clone Sergeant"],
    "Captain Antilles": ["Captain Antilles", "Antilles"],
    "Alderaan Trooper": ["Alderaan Trooper"],
    "Clone Commander Thire": ["Clone Commander Thire", "Commander Thire"],
    "Worker Droid": ["Worker Droid"],
    "Qui-Gon Jinn": ["Qui-Gon", "Qui-Gon Jinn"],
    "Medical Droid": ["Medical Droid"],
    "Polis Medic": ["Polis Medic"],
    "Tarkin": ["Governor Tarkin", "Tarkin"],
    "Queen of Alderaan": ["Queen of Alderaan"],
    "Shaak Ti": ["Shaak Ti"],
    "Baron Papanoida": ["Baron Papanoida", "Papanoida"],
    
}


e4_people = {
    "Luke Skywalker": ["Luke", "Luke Skywalker", "kid", "Master Luke", "Red Five", "Skywalker"],
    "Biggs Darklighter": ["Biggs", "Biggs Darklighter", "Red Three"],
    "Obi-Wan Kenobi": ["Old Ben", "Ben Kenobi", "Old Man", "Kenobi", "Obi-Wan", "Obi-Wan Kenobi", "Ben"],
    "Chewbacca": ["Chewbacca", "Chewie", "Wookie"],
    "Han Solo": ["Han", "Solo", "Han Solo"],
    "Princess Leia": ["Leia", "Princess Leia", "Your Worshipfulness", "Your Highness", "Princess", "Leia Organa"],
    "C-3P0": ["Threepio", "C-3PO", "See-Threepio"],
    "R2-D2": ["Artoo", "R2-D2", "Artoo Deetoo", "Little Artoo"],
    "Jawa": ["Jawa", "Jawas"],
    "Owen Lars": ["Owen", "Owen Lars", "Uncle Owen"],
    "Beru Lars": ["Beru", "Aunt Beru", "Beru Lars"],
    "Tusken Raider": ["Sand People", "Tuskens", "Tusken Raider", "Tusken", "Raider", "Tusken Raiders"],
    "Darth Vader": ["Lord Vader", "Darth Vader", "Vader", "Darth"],
    "Red Seven": ["Red Seven"],
    "Wedge Antilles": ["Wedge", "Wedge Antilles", "Red Two"],
    "Red Ten": ["Red Ten"],
    "Deak": ["Deak"],
    "General Dodonna": ["General Dodonna", "Dodonna"],
    "Captain Antilles": ["Captain Antilles"],
    "Wormie": ["Wormie"],
    "Tarkin": ["Grand Moff Tarkin", "Governor Tarkin", "Tarkin"],
    "Gold Five": ["Gold Five"],
    "Red Leader": ["Red Leader"],
    "Rebel Officer": ["Rebel Officer"],
    "Jabba the Hutt": ["Jabba", "Jabba the Hutt"],
    "Camie": ["Camie"],
    "Stormtrooper": ["Troops", "stormtrooper", "stormtroopers", "trooper"],
    "Fixer": ["Fixer"],
    "Windy": ["Windy"],
    "Rebel Trooper": ["Rebel", "Rebels", "Rebel Trooper", "Rebel Troops"],
    "Greedo": ["Greedo"],
    "Porkins": ["Porkins", "Red Six"],
    "Red Nine": ["Red Nine"],
    "Red Eleven": ["Red Eleven"],
    "Admiral Motti": ["Admiral Motti", "Motti"],
    "Commander Tagge": ["Commander Tagge", "Tagge"],
    "Bartender": ["Bartender"],
    "Ponda Babba": ["Creature"],
    "Doctor Evezan": ["Human"],
    "Imperial Officer": ["Imperial Officer"],
    "Gold Two": ["Gold Two"],
    
}


e5_people = {
    "Luke Skywalker": ["Luke", "Luke Skywalker", "kid", "boy", "Master Luke", "Skywalker", "Rogue leader Luke", "Rogue leader", "Opposite Luke", "Again Luke"],
    "Obi-Wan Kenobi": ["Old Ben", "Ben Kenobi", "Old Man", "Kenobi", "Obi-Wan", "Obi-Wan Kenobi", "Ben"],
    "Chewbacca": ["Chewbacca", "Chewie", "Wookie"],
    "Han Solo": ["Han", "Solo", "Han Solo", "Captain Solo", "Scoundrel"],
    "Princess Leia": ["Leia", "Princess Leia", "Your Worshipfulness", "Your Highness", "Princess", "Leia Organa", "your Worship"],
    "C-3P0": ["Threepio", "C-3PO", "See-Threepio"],
    "R2-D2": ["Artoo", "R2-D2", "Artoo Deetoo", "Little Artoo", "Nearby Artoo"],
    "Darth Vader": ["Lord Vader", "Darth Vader", "Vader", "Darth", "Father", "Piett Lord Vader"],
    "Wedge Antilles": ["Wedge", "Wedge Antilles", "Rogue Three"],
    "Rebel Officer": ["Rebel Officer", "Rebel lieutenant", "deck officer", "senior controller", "controller","Rebel Captain", "trench officer"],
    "Jabba the Hutt": ["Jabba", "Jabba the Hutt", "Hutt"],
    "Stormtrooper": ["Troops", "stormtrooper", "stormtroopers", "trooper", "man's voice"],
    "Rebel Trooper": ["Rebel", "Rebels", "Rebel Trooper", "Rebel Troops", "Rebel Fighter", "announcer"],
    "Imperial Officer": ["Imperial Officer", "commander", "bunker commander", "Imperial soldier", "Imperial controller", "commander"],
    "Lando Calrissian": ["Lando", "Lando Calrissian", "Calrissian", "Gold Leader", "administrator", "Lando Lando"],
    "Admiral Piett": ["Admiral Piett", "Piett", "Captain Piett", "Piett Yes", "Piett Admiral", "Piett bounty", "Piett Thank", "Piett Right", "Piett Alert"],
    "Boba Fett": ["Boba Fett", "Boba", "bounty hunter", "Fett"],
    "Yoda": ["Yoda", "creature", "Jedi Master"],
    "Palpatine": ["Emperor", "Palpatine"],
    "Rebel Pilot": ["Rebel Pilot", "Y-Wing Pilot", "Pilot", "X-wings pilots"],
    "Captain Needa": ["Captain Needa", "Needa"],
    "Major Derlin": ["Derlin", "Major Derlin", "Major"],
    "Janson": ["Janson", "Janson Cable"],
    "Zev": ["Rogue Two", "Zev"],
    "Dack Ralter": ["Dack Ralter", "Dack"],
    "General Rieekan": ["Rieekan", "General Rieekan"],
    "Imperial Probe Droid": ["probe robot", "probe droid"],
    "Admiral Ozzel": ["Ozzel", "Admiral Ozzel"],
    "General Veers": ["Veers", "General Veers"],
    "Medical Droid": ["medical droid"],
    "Hobbie": ["Hobbie"],
    "Walker Pilot": ["walker pilot"],
    "IG-88": ["IG-88"],
    "Dengar": ["Dengar"],
    "Zuckuss": ["Zuckuss"],
    "Bossk": ["Bossk"],
    "4-LOM": ["bounty hunters"],
    "Lobot": ["Lobot"],
    "Ugnaught": ["Ugnaught"],
    "Bespin Guards": ["Bespin Guards"],
    
    
}

e6_people = {
    "Luke Skywalker": ["Luke", "Luke Skywalker", "kid", "Master Luke", "Red Five", "Skywalker", "Jedi Knight"],
    "Obi-Wan Kenobi": ["Old Ben", "Ben Kenobi", "Old Man", "Kenobi", "Obi-Wan", "Obi-Wan Kenobi", "Ben"],
    "Chewbacca": ["Chewbacca", "Chewie", "Wookie"],
    "Han Solo": ["Han", "Solo", "Han Solo", "Captain Solo", "Again Han"],
    "Princess Leia": ["Leia", "Princess Leia", "Your Worshipfulness", "Your Highness", "Princess", "Leia Organa", "Boushh"],
    "C-3P0": ["Threepio", "C-3PO", "See-Threepio", "Seethreepiowha"],
    "R2-D2": ["Artoo", "R2-D2", "Artoo Deetoo", "Little Artoo"],
    "Jawa": ["Jawa", "Jawas"],
    "Tusken Raider": ["Sand People", "Tuskens", "Tusken Raider", "Tusken", "Raider", "Tusken Raiders"],
    "Darth Vader": ["Lord Vader", "Darth Vader", "Vader", "Darth", "Father"],
    "Wedge Antilles": ["Wedge", "Wedge Antilles", "Red Leader"],
    "Rebel Officer": ["Rebel Officer"],
    "Jabba the Hutt": ["Jabba", "Jabba the Hutt", "Hutt"],
    "Stormtrooper": ["Troops", "stormtrooper", "stormtroopers", "trooper"],
    "Rebel Trooper": ["Rebel", "Rebels", "Rebel Trooper", "Rebel Troops"],
    "Imperial Officer": ["Imperial Officer", "commander", "bunker commander", "Imperial soldier"],
    "Lando Calrissian": ["Lando", "Lando Calrissian", "Calrissian", "Gold Leader"],
    "Admiral Piett": ["Admiral Piett", "Piett", "PIETT Hold", "PIETT Shall", "PIETT Intensify"],
    "Boba Fett": ["Boba Fett", "Boba"],
    "Logray": ["Logray"],
    "Chief Chirpa": ["Chief Chirpa"],
    "Ewoks": ["Ewoks"],
    "Wicket": ["Wicket"],
    "Nien Nunb": ["Nien Nunb"],
    "Yoda": ["Yoda"],
    "General Madine": ["Madine", "General Madine"],
    "Mon Mothma": ["Mon Mothma", "Mon", "Mothma"],
    "Paploo": ["Paploo"],
    "Admiral Ackbar": ["Ackbar", "Admiral Ackbar"],
    "Bib Fortuna": ["Bib", "Bib Fortuna", "Fortuna"],
    "Shuttle Captain": ["Shuttle Captain"],
    "Moff Jerjerrod": ["Jerjerrod", "Moff Jerjerrod"],
    "Palpatine": ["Emperor", "Palpatine"],
    "Gamorrean Guard": ["Gamorrean Guard", "Gamorrean"],
    "Salacious Crumb": ["Salacious Crumb", "Crumb", "Salacious"],
    "Oola": ["Oola"],
    "EV-9D9": ["EV-9D9", "Ninedenine"],
    "Rancor Keeper": ["Keeper"],
    "Flight Controller": ["Controller"],
    "Imperial Scout Trooper": ["Imperial scouts", "scout", "biker scouts"],
    "Royal Guards": ["Imperial Guards", "red-cloaked guards"],
    "Teebo": ["Teebo"],
    "Gray Leader": ["Grey Leader"],
    "Green Leader": ["Green Leader"],
    "Red Leader": ["Red Leader"],
    "Rebel Pilot": ["Rebel Pilot", "Y-Wing Pilot", "Pilot", "X-wings pilots"],
    "Red Two": ["Red Two"],
    "Red Three": ["Red Three"],
    "Walker Pilot": ["Walker Pilot"],
    "Anakin Skywalker": ["Anakin Skywalker", "Anakin"],
    
}

e_people = merge_dicts(e12_people, e3_people, e4_people, e5_people, e6_people)

print(e_people)


{'Obi-Wan Kenobi': ['Obi-Wan', 'Kenobi', 'Master Jedi', 'Obi-Wan Kenobi', 'Master Obi-Wan Kenobi', 'Master Kenobi', 'Obi Wan', 'Old Ben', 'Ben Kenobi', 'Old Man', 'Ben'], 'Anakin Skywalker': ['boy', 'Anakin', 'Anakin Skywalker', 'Skywalker', 'Annie', 'Anakin SKywalker', 'Young Skywalker', 'my young padawan', 'Master Skywalker', 'Master Anakin', 'Helloo Annie'], 'Jar Jar Binks': ['Jar Jar', 'Binks', 'Jar Jar Binks', 'Jaja Binkss', 'Bombad general', 'Binkss'], 'Nute Gunray': ['Viceroy', 'Gunray', 'Nute', 'Nute Gunray', 'Viceroy Gunray'], 'Padme Amidala': ['Padme Naberrie', 'Padme Amidala', 'Padme'], 'Yoda': ['Master Yoda', 'Yoda', 'creature', 'Jedi Master'], 'Mace Windu': ['Master Windu', 'Mace', 'Windu', 'Mace Windu'], 'Battle Droid': ['battle droid', 'battle droids', 'OOM-9', 'droids', 'droid sergeant', 'battle droid captain', 'guard droid', 'Roger', 'command droids'], 'Watto': ['Watto'], 'Palpatine': ['Senator Palpatine', 'Darth Sidious', 'Palpatione', 'Sidious', 'Palpatine', 'Chancel

In [4]:
print(set_all)

{'Green Leader', 'Qui-Gon Jinn', 'Alderaan Trooper', 'Chi Eekway', 'Oola', 'Sifo-Dyas', 'Yoda', 'Royal Guards', 'Fang Zar', 'Owen Lars', 'Red Two', 'Dexter Jettster', 'Tion Medon', 'Captain Antilles', 'Rebel Pilot', 'Admiral Piett', 'Nee Avalar', 'Wrangler', 'Imperial Officer', 'Cliegg Lars', 'R2-D2', 'TC-14', 'AT-ST Clone Sergeant', 'Ryoo', 'Flight Controller', 'Boss Nass', 'Ruwee', 'Eirtae', 'Teebo', 'Commander Tagge', 'Red Leader', 'General Dodonna', 'Tey How', 'EV-9D9', 'Biggs Darklighter', 'Fode/Beed', 'Wookie Warrior', 'Major Derlin', 'Agen Kolar', 'Ewoks', 'Windy', 'Logray', 'Dorme', 'Red Three', 'Bespin Guards', 'Wicket', 'Taun We', 'Wedge Antilles', 'Jango Fett', 'Sith Probe Droid', 'Shu Mai', 'Jawa', 'IG-88', 'Boles Roor', 'Nute Gunray', 'Amee', 'Sola', 'Paploo', 'Princess Leia', 'Bossk', 'Boba Fett', 'Sun Rit', 'Bib Fortuna', 'Saesee Tinn', 'Gray Leader', 'Chief Chirpa', 'Bartender', 'Chancellor Valorum', 'Wald', 'Bana Breemu', 'Rancor Keeper', 'Pit Droid', 'Lott Dod', 'Gene

In [ ]:
set_e1 = {"Obi-Wan Kenobi", "Qui-Gon Jinn", "TC-14", "Jabba the Hutt", "Anakin Skywalker", "Jar Jar Binks", \
              "Captain Tarpals", "Boss Nass", "Nute Gunray", "Republic Captain", "Captain Panaka", "Darth Maul", \
              "Padme Amidala", "Yoda", "Mace Windu", "Dofine", "Battle Droid", "Rune", "Tey How", "Sebulba", "Watto", "Eirtae", \
              "Yane", "Rabe", "Sache", "Palpatine", "Sio Bibble", "Chancellor Valorum", "R2-D2", "C-3PO", "Ric Olie", "Queen", \
              "Jira", "Shmi", "Kitster", "Seek", "Amee", "Wald", "Fode/Beed", "Ben Quadrinaros", "Boles Roor", "Gasgano", \
              "Mawhonic", "Bail Organa", "Gardulla", "Xelbree", "Fanta", "Ody Mandrell", "Terter", "Sith Probe Droid", "Greedo", \
              "Ki-Adi-Mundi", "Lott Dod", "Aks Moe", "Mas Amedda", "Gungan Warrior", "Bravo Two", "Bravo Three", "Destroyer Droids", "Pit Droid"}

set_e2 = {"Obi-Wan Kenobi", "Anakin Skywalker", "Jar Jar Binks", "Nute Gunray", "Padme Amidala", "Yoda", "Mace Windu",\
              "Battle Droid", "Watto", "Palpatine", "Sio Bibble", "R2-D2", "C-3PO", "Shmi", "Bail Organa", "Ki-Adi-Mundi",\
              "Mas Amedda", "Destroyer Droids", "Cliegg Lars", "Owen Lars", "Beru Lars", "Sifo-Dyas", "Count Dooku",\
              "Jango Fett", "Zam Wessel", "Boba Fett", "Tusken Raider", "Dorme", "Captain Typho", "Corde", "Ask Aak", "Darsana",\
              "Orn Free Taa", "Plo Koon", "Royal Guards", "Dar Wac", "Elan Sleazebaggano", "PK-4", "Dexter Jettster",\
              "Queen Jamillia", "Jocasta Nu", "Ryoo", "Pooja", "Sola", "Jobal", "Ruwee", "Jedi Child Jack", "Jedi Child May",\
              "Taun We", "Lama Su", "R4-D17", "Pit Droid", "Clone Troopers", "Geonosian", "Poggle the Lesser", "Sun Rit",\
              "Po Nudu", "Tessek", "Toonbuck Toora", "Shu Mai", "Passel Argente", "Wat Tambor", "San Hill", "Super Battle Droids",\
              "Youngling"}

set_e3 = {"Obi-Wan Kenobi", "Anakin Skywalker", "Jar Jar Binks", "Nute Gunray", "Padme Amidala", "Yoda", "Mace Windu",\
              "Battle Droid", "Palpatine", "Sio Bibble", "R2-D2", "C-3PO", "Bail Organa", "Ki-Adi-Mundi", "Mas Amedda",\
              "Destroyer Droids", "Owen Lars", "Beru Lars", "Count Dooku", "Captain Typho", "Plo Koon", "Royal Guards",\
              "Dar Wac", "Queen Jamillia", "R4-D17", "Clone Troopers", "Poggle the Lesser", "Sun Rit", "Po Nudu", "Tessek",\
              "Toonbuck Toora", "Shu Mai", "Passel Argente", "Wat Tambor", "San Hill", "Super Battle Droids", "General Grievous",\
              "Buzz Droid", "Odd Ball", "Clone Pilot", "Nemoidian Captain", "Bodyguard", "Fireship Pilot", "Luke Skywalker",\
              "Princess Leia", "Master Vos", "Mon Mothma", "Fang Zar", "Terr Taneel", "Giddean Danu", "Eeth Koth", "Elle", "Motee",\
              "Sly-Moore", "Darth Plagueis", "Agen Kolar", "Chewbacca", "Wookie Warrior", "Bana Breemu", "Chi Eekway",\
              "Clone Commander Cody", "Tion Medon", "R4-G9", "Wrangler", "Rune Haako", "Nee Avalar", "Maledee",\
              "Sweitt Concorkill", "Aayla Secura", "Saesee Tinn", "Kit Fisto", "Darth Vader", "Clone Commander Bacara",\
              "Barriss Offee", "Stass Allie", "Clone Commander Gree", "Tarfful", "Youngling", "Zett Jukassa",\
              "Clone Commander Appo", "Seeker Drone", "AT-ST Clone Sergeant", "Captain Antilles", "Alderaan Trooper",\
              "Clone Commander Thire", "Worker Droid", "Qui-Gon Jinn", "Medical Droid", "Polis Medic", "Tarkin",\
              "Shaak Ti", "Queen of Alderaan", "Baron Papanoida"}


set_e4 = {"Luke Skywalker", "Biggs Darklighter", "Obi-Wan Kenobi", "Chewbacca", "Han Solo", "Princess Leia", "C-3PO", "R2-D2", \
              "Jawa", "Owen Lars", "Beru Lars", "Tusken Raider", "Darth Vader", "Red Seven", "Wedge Antilles", "Red Ten", "Deak", \
              "General Dodonna", "Captain Antilles", "Wormie", "Tarkin", "Gold Five", "Red Leader", "Rebel Officer", "Jabba the Hutt" \
              "Camie", "Stormtrooper", "Fixer", "Windy", "Rebel Trooper", "Greedo", "Red Nine", "Red Eleven", "Admiral Motti", \
              "Commander Tagge", "Bartender", "Ponda Babba", "Doctor Evezan", "Imperial Officer", "Gold Two"}

set_e5 = {"Luke Skywalker", "Obi-Wan Kenobi", "Chewbacca", "Han Solo", "Princess Leia", "C-3PO", "R2-D2", "Darth Vader",\
              "Wedge Antilles", "Rebel Officer", "Jabba the Hutt", "Stormtrooper", "Rebel Trooper", "Imperial Officer",\
              "Lando Calrissian", "Admiral Piett", "Boba Fett", "Yoda", "Palpatine", "Rebel Pilot", "Captain Needa",\
              "Major Derlin", "Janson", "Zev", "Dack Ralter", "General Rieekan", "Imperial Probe Droid", "Admiral Ozzel",\
              "General Veers", "Medical Droid", "Hobbie", "Walker Pilot", "IG-88", "Dengar", "Zuckuss", "Bossk", "4-LOM",\
              "Lobot", "Ugnaught", "Bespin Guards"}

set_e6 = {"Luke Skywalker", "Obi-Wan Kenobi", "Chewbacca", "Han Solo", "Princess Leia", "C-3PO", "R2-D2", "Jawa",\
              "Tusken Raider", "Darth Vader", "Wedge Antilles", "Rebel Officer", "Jabba the Hutt", "Stormtrooper",\
              "Imperial Officer", "Lando Calrissian", "Admiral Piett", "Boba Fett", "Logray", "Chief Chirpa", "Ewoks",\
              "Wicket", "Nien Nunb", "Yoda", "General Madine", "Mon Mothma", "Paploo", "Admiral Ackbar", "Bib Fortuna",\
              "Shuttle Captain", "Moff Jerjerrod", "Palpatine", "Gamorrean Guard", "Salacious Crumb", "Oola", "EV-9D9",\
              "Rancor Keeper", "Flight Controller", "Imperial Scout Trooper", "Royal Guards", "Teebo", "Gray Leader",\
              "Green Leader", "Rebel Pilot", "Red Two", "Red Three", "Walker Pilot", "Anakin Skywalker"}

set_all = set_e1.union(set_e2).union(set_e3).union(set_e4).union(set_e5).union(set_e6)